In [108]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine, correlation
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing:

In [275]:
anime=pd.read_csv(r"C:\Users\disha\Downloads\Recommendation System\Recommendation System\anime.csv")
anime

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [276]:
print(anime.head())

   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  


In [277]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [278]:
anime=anime[['anime_id','name','genre','type','episodes','rating','members']]

In [279]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [280]:
anime.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [281]:
anime.dropna(inplace=True)

In [282]:
anime.duplicated().sum()

0

In [283]:
anime.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
12289    False
12290    False
12291    False
12292    False
12293    False
Length: 12017, dtype: bool

In [284]:
anime.iloc[0].genre

'Drama, Romance, School, Supernatural'

# Feature Extraction:

In [285]:
anime['tags']=anime['name']+anime['genre']+anime['type']


In [286]:
anime

,anime_id,name,genre,type,episodes,rating,members,tags
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,"Kimi no Na wa.Drama, Romance, School, Supernat..."
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,"Fullmetal Alchemist: BrotherhoodAction, Advent..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,"Gintama°Action, Comedy, Historical, Parody, Sa..."
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"Steins;GateSci-Fi, ThrillerTV"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"Gintama&#039;Action, Comedy, Historical, Parod..."
...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,Toushindai My Lover: Minami tai Mecha-MinamiHe...
12290,5543,Under World,Hentai,OVA,1,4.28,183,Under WorldHentaiOVA
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219,Violence Gekiga David no HoshiHentaiOVA
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,Violence Gekiga Shin David no Hoshi: Inma Dens...


In [287]:
anime_df=anime[['anime_id','name','episodes','rating','members','tags']]
anime_df

,anime_id,name,episodes,rating,members,tags
0,32281,Kimi no Na wa.,1,9.37,200630,"Kimi no Na wa.Drama, Romance, School, Supernat..."
1,5114,Fullmetal Alchemist: Brotherhood,64,9.26,793665,"Fullmetal Alchemist: BrotherhoodAction, Advent..."
2,28977,Gintama°,51,9.25,114262,"Gintama°Action, Comedy, Historical, Parody, Sa..."
3,9253,Steins;Gate,24,9.17,673572,"Steins;GateSci-Fi, ThrillerTV"
4,9969,Gintama&#039;,51,9.16,151266,"Gintama&#039;Action, Comedy, Historical, Parod..."
...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,1,4.15,211,Toushindai My Lover: Minami tai Mecha-MinamiHe...
12290,5543,Under World,1,4.28,183,Under WorldHentaiOVA
12291,5621,Violence Gekiga David no Hoshi,4,4.88,219,Violence Gekiga David no HoshiHentaiOVA
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,1,4.98,175,Violence Gekiga Shin David no Hoshi: Inma Dens...


In [288]:
cv=CountVectorizer(max_features=5000, stop_words='english')
cv.fit_transform(anime_df['tags']).toarray().shape

(12017, 5000)

In [289]:
vectors=cv.fit_transform(anime_df['tags']).toarray()
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [290]:
len(cv.get_feature_names())

5000

In [291]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
        return"".join(y)

In [292]:
anime_df['tags']=anime_df['tags'].apply(stem)

# Recommendation System:


In [293]:
cosine_similarity(vectors)

array([[1.        , 0.13363062, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.13363062, 1.        , 0.11785113, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.11785113, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.70710678,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.70710678, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [294]:
cosine_similarity(vectors).shape

(12017, 12017)

In [295]:
similarity=cosine_similarity(vectors)

In [296]:
similarity[0]

array([1.        , 0.13363062, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [297]:
similarity[0].shape

(12017,)

In [298]:
sorted(list(enumerate(similarity[0])),reverse=True, key=lambda x:x[1])[1:8]

[(16, 0.5714285714285713),
 (158, 0.5345224838248487),
 (1111, 0.5345224838248487),
 (1494, 0.5345224838248487),
 (88, 0.50709255283711),
 (404, 0.5039526306789696),
 (764, 0.5039526306789696)]

In [319]:
def recommend(anime):
    if anime not in anime_df['name'].values:
        print(f"No anime with name '{anime}' found.")
        return []
    
    anime_index = anime_df[anime_df['name'] == anime].index[0]
    distances = similarity[anime_index]
    anime_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:8]
    
    recommendations = [anime_df.iloc[i[0]]['name'] for i in anime_list]
    
    return recommendations


In [320]:
recommend('Fullmetal Alchemist: Brotherhood')

['Fullmetal Alchemist',
 'Fullmetal Alchemist: The Sacred Star of Milos',
 'Fullmetal Alchemist: Brotherhood Specials',
 'Fullmetal Alchemist: The Conqueror of Shamballa',
 'Fullmetal Alchemist: Reflections',
 'Tide-Line Blue',
 'Arion']

In [321]:
recommend('Steins;Gate')

['Steins;Gate: Oukoubakko no Poriomania',
 'Zankyou no Terror',
 'Steins;Gate Movie: Fuka Ryouiki no Déjà vu',
 'Gankutsuou',
 'Steins;Gate: Kyoukaimenjou no Missing Link - Divide By Zero',
 'Steins;Gate: Soumei Eichi no Cognitive Computing',
 'Element Hunters']